# Grid Search hyper parameters for improved SRST

1. Import dependencies

In [ ]:
import random
import torch
import numpy as np
import pandas as pd
import time
import glob
import os

from src.util.torch import resolve_torch_device
from src.data.indian_pines import load_indian_pines
from src.util.hsi import sample_fraction_from_segmentation
from src.definitions import GREED_SEARCH_FOLDER
from src.model.grid_search import GridSearch
from src.model.improved_spatial_regulated_self_training_grid_search import (
    ImprovedSpatialRegulatedSelfTrainingPipelineGridSearchAdapter,
)

2. Prepare env

In [2]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

generator = torch.Generator()
generator.manual_seed(random_seed)

In [3]:
f"Device is {device}"

'Device is cuda'

# Indian pines (Cluster exponential decay)

0. Set params

In [4]:
examples_per_class = 0.7
epoch_seconds = 1745282320
run_name = f"improved-indian-pines-cluster-exponential-decay-{epoch_seconds}"

In [5]:
cpu_count = 4

f"Setting num_workers to {cpu_count}"

'Setting num_workers to 4'

1. Load dataset

In [6]:
image, labels = load_indian_pines()

In [7]:
num_classes = len(np.unique(labels))

f"Number of classes {num_classes}"

'Number of classes 17'

In [8]:
masked_labels = sample_fraction_from_segmentation(labels, examples_per_class)

2. Train model

In [9]:
params = {
    "input_channels": [50, 125, 200],
    "learning_rate": [1e-3, 1e-4],
    "patch_size": [9],
    "num_epochs": [11],
    "feature_extractor_epochs": [1, 9, 11],
    "semantic_threshold": [0.5, 0.6, 0.8],
    "lambda_v": [0.07, 0.3, 0.49],
    "k_star": [num_classes * 3],
    "batch_size": [64],
}

adapter = ImprovedSpatialRegulatedSelfTrainingPipelineGridSearchAdapter(
    params,
    image,
    masked_labels,
    labels,
    num_classes,
    device,
    random_seed,
    generator,
)

In [10]:
log_dir = GREED_SEARCH_FOLDER / run_name

In [11]:
search = GridSearch(
    adapter=adapter,
    optimize_metric="kappa_score",
    log_dir=log_dir,
    num_workers=cpu_count,
)

In [ ]:
_, best_params, best_score = search.run()

In [ ]:
print("Best Params:", best_params)
print("Best Score:", best_score)

3. Training results

In [12]:
csv_files = glob.glob(os.path.join(log_dir, "*.csv"))

report = pd.concat([pd.read_csv(f) for f in csv_files])

report.head()

,input_channels,learning_rate,patch_size,num_epochs,feature_extractor_epochs,semantic_threshold,lambda_v,k_star,batch_size,overall_accuracy,average_accuracy,kappa_score,f1_score,best_iteration,best_kappa_score
0,200,0.001,9,11,9,0.6,0.49,51,64,0.275979,0.269822,0.172599,0.231070,0,0.892375
1,200,0.001,9,11,9,0.8,0.07,51,64,0.058824,0.512533,0.000000,0.347351,0,0.000000
2,200,0.001,9,11,9,0.8,0.30,51,64,0.064961,0.125279,0.008359,0.132326,3,0.114542
3,200,0.001,9,11,9,0.8,0.49,51,64,0.218416,0.236195,0.154109,0.259043,0,0.893830
4,200,0.001,9,11,11,0.5,0.07,51,64,0.058824,0.512533,0.000000,0.347351,0,0.000000


In [13]:
len(report)

162

In [14]:
report.sort_values("kappa_score", ascending=False).head()

,input_channels,learning_rate,patch_size,num_epochs,feature_extractor_epochs,semantic_threshold,lambda_v,k_star,batch_size,overall_accuracy,average_accuracy,kappa_score,f1_score,best_iteration,best_kappa_score
33,200,0.0001,9,11,11,0.5,0.49,51,64,0.576155,0.861260,0.789690,0.848080,1,0.900036
36,200,0.0001,9,11,11,0.6,0.49,51,64,0.582670,0.854174,0.777485,0.845396,1,0.907199
27,200,0.0001,9,11,9,0.6,0.49,51,64,0.520374,0.852985,0.775552,0.835933,1,0.900798
30,200,0.0001,9,11,9,0.8,0.49,51,64,0.514440,0.851082,0.773029,0.833396,1,0.897675
24,200,0.0001,9,11,9,0.5,0.49,51,64,0.578732,0.845707,0.763662,0.837862,1,0.900191
